In [1]:
import requests
import json
import pandas as pd

In [2]:
# api to get free currency rate live
api = requests.get('http://apilayer.net/api/live?access_key=2abd4830413987f2f83863e4461c26a7&source=USD&format=1')

In [3]:
response_text = api.json()
response_json = json.dumps(response_text, indent=4)
# print(response_json)

Source: USD
Timestamp: provided

In [4]:
api_timestamp= response_text.get('timestamp')

In [5]:
# create df to store source and destination currency exchange rate along with timestamp
currencies_df = pd.read_json(response_json)

<ipython-input-5-d982e6ecba47>:2: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  currencies_df = pd.read_json(response_json)


In [6]:
currencies_df.head()

,success,terms,privacy,timestamp,source,quotes
USDAED,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,2024-09-29 03:44:15,USD,3.673042
USDAFN,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,2024-09-29 03:44:15,USD,68.000368
USDALL,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,2024-09-29 03:44:15,USD,88.250403
USDAMD,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,2024-09-29 03:44:15,USD,387.320403
USDANG,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,2024-09-29 03:44:15,USD,1.801995


In [7]:
# prompt: convert index to a column
currencies_df = currencies_df.reset_index()

In [8]:
currencies_df.head()

,index,success,terms,privacy,timestamp,source,quotes
0,USDAED,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,2024-09-29 03:44:15,USD,3.673042
1,USDAFN,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,2024-09-29 03:44:15,USD,68.000368
2,USDALL,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,2024-09-29 03:44:15,USD,88.250403
3,USDAMD,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,2024-09-29 03:44:15,USD,387.320403
4,USDANG,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,2024-09-29 03:44:15,USD,1.801995


In [9]:
currencies_df = currencies_df.drop(columns = ['success','terms','privacy'])

In [10]:
currencies_df = currencies_df.rename(columns = {'index': 'target','quotes': 'price'})

In [11]:
currencies_df.head()

,target,timestamp,source,price
0,USDAED,2024-09-29 03:44:15,USD,3.673042
1,USDAFN,2024-09-29 03:44:15,USD,68.000368
2,USDALL,2024-09-29 03:44:15,USD,88.250403
3,USDAMD,2024-09-29 03:44:15,USD,387.320403
4,USDANG,2024-09-29 03:44:15,USD,1.801995


In [12]:
# clean the string naming convention
currencies_df['target'] = currencies_df['target'].str.replace('USD','')

In [14]:
# add usd to usd one row to currencies_df
currencies_df.loc[len(currencies_df.index)] = ['USD',api_timestamp,'USD', 1]
currencies_df.reset_index(inplace=True,drop=True)

In [15]:
currencies_df.head()

,target,timestamp,source,price
0,AED,2024-09-29 03:44:15,USD,3.673042
1,AFN,2024-09-29 03:44:15,USD,68.000368
2,ALL,2024-09-29 03:44:15,USD,88.250403
3,AMD,2024-09-29 03:44:15,USD,387.320403
4,ANG,2024-09-29 03:44:15,USD,1.801995


In [16]:
# method to get the converted currency from any input currency
def convert_currency(input_currency, output_currency, amount):
  output_usd = currencies_df.loc[currencies_df['target'] == output_currency, 'price'].values[0]
  input_usd = currencies_df.loc[currencies_df['target'] == input_currency, 'price'].values[0]
  output_usd = round(float(output_usd),2)
  input_usd = round(float(input_usd),2)
  print(input_usd,output_usd)
  converted_amount = amount * (output_usd / input_usd)
  return round(converted_amount,2)

In [17]:
print(convert_currency('USD', 'USD', 100)) # input, output, price to convert

1.0 1.0
100.0
